In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import os
import seaborn as sns
from matplotlib import rcParams
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Reading weather data
final_weather=pd.read_csv("cleaned_data/weather_data.csv")


In [3]:
#seting county and state as index
df_weather=final_weather.set_index(["County", "State"])
#only lat and lon colums
df_weather=df_weather[["Lon", "Lat", "Max_temp","Min_temp"]]

## Field Crops

In [4]:
#reading all field crops csv for different states data to pandas dataframe
field_crops_data=[]
for i in range(1,8):
    df=pd.read_csv(f'data/field_crops{i}.csv')
    field_crops_data.append(df)
#concating all states datas to one dataframe
field_crops=pd.concat([data for data in field_crops_data], axis=0)

In [5]:
#all columns of dataframe
field_crops.columns

Index(['Program', 'Year', 'Period', 'Week Ending', 'Geo Level', 'State',
       'State ANSI', 'Ag District', 'Ag District Code', 'County',
       'County ANSI', 'Zip Code', 'Region', 'watershed_code', 'Watershed',
       'Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value',
       'CV (%)'],
      dtype='object')

In [6]:
#selecting only important columns
field_crops=field_crops[['County','State','Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value']]

In [7]:
#replacing string (D) with 0 in value columns and converting to float
field_crops['Value']=field_crops['Value'].replace({' (D)':0, ' (Z)':0})
field_crops['Value']=field_crops['Value'].str.replace(',', '').astype(float)

In [8]:
#All field crops names
field_crops_list=field_crops['Commodity'].unique().tolist()


In [9]:
#value for each comodity with county
field_crops_df=field_crops.groupby(['County', 'State', 'Commodity']).agg({'Value':sum})
field_crops_df.reset_index(inplace=True)
field_crops_df

,County,State,Commodity,Value
0,ABBEVILLE,SOUTH CAROLINA,CORN,579.0
1,ABBEVILLE,SOUTH CAROLINA,COTTON,4.0
2,ABBEVILLE,SOUTH CAROLINA,"FIELD CROPS, OTHER",825137.0
3,ABBEVILLE,SOUTH CAROLINA,GRAIN,10021.0
4,ABBEVILLE,SOUTH CAROLINA,HAY,31587.0
...,...,...,...,...
34678,ZIEBACH,SOUTH DAKOTA,PEAS,27952.0
34679,ZIEBACH,SOUTH DAKOTA,SORGHUM,6.0
34680,ZIEBACH,SOUTH DAKOTA,SOYBEANS,341278.0
34681,ZIEBACH,SOUTH DAKOTA,SUNFLOWER,93924215.0


In [10]:
#making state , county captilize
field_crops_df['State']=field_crops_df['State'].str.capitalize()
field_crops_df['County']=field_crops_df['County'].str.capitalize()
field_crops_df['Commodity']=field_crops_df['Commodity'].str.capitalize()

In [11]:
def state_abbr(x):
    return (x.replace( "Alaska", "AK").replace("Alabama", "AL").replace("Arizona", "AZ").replace("Arkansas", "AR").replace("California", "CA").replace("Colorado", "CO").replace("Connecticut", "CT").replace("Delaware", "DE").replace("Florida", "FL").replace("Georgia", "GA").replace("Hawaii", "HI").replace("Idaho", "ID").replace("Illinois", "IL").replace("Indiana", "IN").replace("Iowa", "IA").replace("Kansas", "KS").replace("Kentucky", "KY").replace("Louisiana", "LA").replace("Maine", "ME").replace("Maryland", "MD").replace("Massachusetts", "MA").replace("Michigan", "MI").replace("Minnesota", "MN").replace("Mississippi", "MS").replace("Missouri","MO").replace("Montana", "MT").replace("Nebraska", "NE").replace("Nevada", "NV").replace("New hampshire", "NH").replace("New jersey", "NJ").replace("New mexico", "NM").replace("New york", "NY").replace('North carolina', "NC").replace("North dakota", "ND").replace("Ohio", "OH").replace("Oklahoma", "OK").replace("Oregon", "OR").replace("Pennsylvania", "PA").replace("Rhode island", "RI").replace("South carolina", "SC").replace("South dakota", "SD").replace("Tennessee", "TN").replace("Texas", "TX").replace("Utah", "UT").replace("Vermont", "VT").replace("Virginia", "VA").replace("Washington", "WA").replace("West virginia", "WV").replace("Wisconsin", "WI").replace("Wyoming", "WY"))

In [12]:
field_crops_df['State'] = field_crops_df['State']. apply(state_abbr)
field_crops_df

,County,State,Commodity,Value
0,Abbeville,SC,Corn,579.0
1,Abbeville,SC,Cotton,4.0
2,Abbeville,SC,"Field crops, other",825137.0
3,Abbeville,SC,Grain,10021.0
4,Abbeville,SC,Hay,31587.0
...,...,...,...,...
34678,Ziebach,SD,Peas,27952.0
34679,Ziebach,SD,Sorghum,6.0
34680,Ziebach,SD,Soybeans,341278.0
34681,Ziebach,SD,Sunflower,93924215.0


In [13]:
#setting index county and state
df_field_crops=field_crops_df.set_index(["County", "State"])


In [14]:
#merging with weather to get lat and lon
field_crops_merged = df_field_crops.merge(df_weather, how='inner', left_index=True, right_index=True)
field_crops_merged

Commodity       Value       Lon      Lat  Max_temp  \
County    State                                                                
Abbeville SC                   Corn       579.0  -82.3785  34.1819      7.51   
          SC                 Cotton         4.0  -82.3785  34.1819      7.51   
          SC     Field crops, other    825137.0  -82.3785  34.1819      7.51   
          SC                  Grain     10021.0  -82.3785  34.1819      7.51   
          SC                    Hay     31587.0  -82.3785  34.1819      7.51   
...                             ...         ...       ...      ...       ...   
Ziebach   SD                   Peas     27952.0 -101.4999  44.6055     -5.05   
          SD                Sorghum         6.0 -101.4999  44.6055     -5.05   
          SD               Soybeans    341278.0 -101.4999  44.6055     -5.05   
          SD              Sunflower  93924215.0 -101.4999  44.6055     -5.05   
          SD                  Wheat   7816545.0 -101.4999  44.6055     -5.05   

                 Min_temp  
County    State            
Abbeville SC         3.21  
          SC         3.21  
          SC         3.21  
          SC         3.21  
          SC         3.21  
...                   ...  
Ziebach   SD        -7.36  
          SD        -7.36  
          SD        -7.36  
          SD        -7.36  
          SD        -7.36  

[31830 rows x 6 columns]

In [15]:
#reseting index
field_crops_merged.reset_index(inplace=True)
field_crops_merged

,County,State,Commodity,Value,Lon,Lat,Max_temp,Min_temp
0,Abbeville,SC,Corn,579.0,-82.3785,34.1819,7.51,3.21
1,Abbeville,SC,Cotton,4.0,-82.3785,34.1819,7.51,3.21
2,Abbeville,SC,"Field crops, other",825137.0,-82.3785,34.1819,7.51,3.21
3,Abbeville,SC,Grain,10021.0,-82.3785,34.1819,7.51,3.21
4,Abbeville,SC,Hay,31587.0,-82.3785,34.1819,7.51,3.21
...,...,...,...,...,...,...,...,...
31825,Ziebach,SD,Peas,27952.0,-101.4999,44.6055,-5.05,-7.36
31826,Ziebach,SD,Sorghum,6.0,-101.4999,44.6055,-5.05,-7.36
31827,Ziebach,SD,Soybeans,341278.0,-101.4999,44.6055,-5.05,-7.36
31828,Ziebach,SD,Sunflower,93924215.0,-101.4999,44.6055,-5.05,-7.36


In [16]:
#All field crops names
crops_list=field_crops_merged['Commodity'].unique().tolist()
crops_list

['Corn',
 'Cotton',
 'Field crops, other',
 'Grain',
 'Hay',
 'Hay & haylage',
 'Haylage',
 'Oats',
 'Peas',
 'Rye',
 'Soybeans',
 'Sunflower',
 'Wheat',
 'Barley',
 'Mint',
 'Rapeseed',
 'Sorghum',
 'Beans',
 'Chickpeas',
 'Grasses',
 'Grasses & legumes totals',
 'Grasses & legumes, other',
 'Legumes',
 'Mustard',
 'Sugarbeets',
 'Triticale',
 'Tobacco',
 'Herbs',
 'Millet',
 'Emmer & spelt',
 'Popcorn',
 'Canola',
 'Flaxseed',
 'Maple syrup',
 'Buckwheat',
 'Camelina',
 'Hops',
 'Peanuts',
 'Wild rice',
 'Sugarcane',
 'Rice',
 'Sesame',
 'Taro',
 'Safflower',
 'Guar',
 'Miscanthus',
 'Lentils',
 'Switchgrass',
 'Dill',
 'Jojoba']

In [17]:
#saving combined data as csv 
field_crops_merged.to_csv("cleaned_data/field_crops_merged.csv",encoding="utf-8", index=False)

In [18]:
field_crops_merged.to_json("cleaned_data/field_crops_merged.json", orient="index")

## Vegetables

In [19]:
#reading all vegetables csv for different states data to pandas dataframe
vegetables_data=[]
for i in range(8):
    df=pd.read_csv(f'data/vegetables{i}.csv')
    vegetables_data.append(df)
#concating all states datas to one dataframe
vegetables=pd.concat([data for data in vegetables_data], axis=0)

In [20]:
#selecting only important columns
vegetables=vegetables[['County','State','Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value']]

In [21]:
#replacing string (D) with 0 in value columns and converting to float
vegetables['Value']=vegetables['Value'].replace({' (D)':0, ' (Z)':0})
vegetables['Value']=vegetables['Value'].str.replace(',', '').astype(float)

In [22]:
vegetables.head()

,County,State,Commodity,Data Item,Domain,Domain Category,Value
0,AUTAUGA,ALABAMA,BEANS,"BEANS, GREEN, LIMA - ACRES HARVESTED",TOTAL,NOT SPECIFIED,NaN
1,AUTAUGA,ALABAMA,BEANS,"BEANS, GREEN, LIMA - OPERATIONS WITH AREA HARV...",TOTAL,NOT SPECIFIED,1.0
2,AUTAUGA,ALABAMA,BEANS,"BEANS, GREEN, LIMA, FRESH MARKET - ACRES HARVE...",TOTAL,NOT SPECIFIED,NaN
3,AUTAUGA,ALABAMA,BEANS,"BEANS, GREEN, LIMA, FRESH MARKET - OPERATIONS ...",TOTAL,NOT SPECIFIED,1.0
4,AUTAUGA,ALABAMA,BEANS,"BEANS, SNAP - ACRES HARVESTED",TOTAL,NOT SPECIFIED,NaN


In [23]:
#value  for each comodity with county
vegetables_df=vegetables.groupby(['County', 'State', 'Commodity']).agg({'Value':sum})
vegetables_df.reset_index(inplace=True)
vegetables_df

,County,State,Commodity,Value
0,ABBEVILLE,SOUTH CAROLINA,BEANS,16.0
1,ABBEVILLE,SOUTH CAROLINA,BROCCOLI,12.0
2,ABBEVILLE,SOUTH CAROLINA,CABBAGE,8.0
3,ABBEVILLE,SOUTH CAROLINA,CARROTS,2.0
4,ABBEVILLE,SOUTH CAROLINA,CAULIFLOWER,8.0
...,...,...,...,...
54308,ZAVALA,TEXAS,CUCUMBERS,186.0
54309,ZAVALA,TEXAS,MELONS,2.0
54310,ZAVALA,TEXAS,ONIONS,4.0
54311,ZAVALA,TEXAS,SPINACH,5.0


In [24]:
#making state , county captilize
vegetables_df['State']=vegetables_df['State'].str.capitalize()
vegetables_df['County']=vegetables_df['County'].str.capitalize()
vegetables_df['Commodity']=vegetables_df['Commodity'].str.capitalize()

In [25]:
vegetables_df['State'] = vegetables_df['State']. apply(state_abbr)
vegetables_df

,County,State,Commodity,Value
0,Abbeville,SC,Beans,16.0
1,Abbeville,SC,Broccoli,12.0
2,Abbeville,SC,Cabbage,8.0
3,Abbeville,SC,Carrots,2.0
4,Abbeville,SC,Cauliflower,8.0
...,...,...,...,...
54308,Zavala,TX,Cucumbers,186.0
54309,Zavala,TX,Melons,2.0
54310,Zavala,TX,Onions,4.0
54311,Zavala,TX,Spinach,5.0


In [26]:
#setting index county and state
df_vegetables=vegetables_df.set_index(["County", "State"])


In [27]:
#merging with weather to get lat and lon
vegetables_merged = df_vegetables.merge(df_weather, how='inner', left_index=True, right_index=True)
vegetables_merged

Commodity      Value      Lon      Lat  Max_temp  \
County    State                                                            
Abbeville SC                Beans       16.0 -82.3785  34.1819      7.51   
          SC             Broccoli       12.0 -82.3785  34.1819      7.51   
          SC              Cabbage        8.0 -82.3785  34.1819      7.51   
          SC              Carrots        2.0 -82.3785  34.1819      7.51   
          SC          Cauliflower        8.0 -82.3785  34.1819      7.51   
...                           ...        ...      ...      ...       ...   
Zavala    TX            Cucumbers      186.0 -99.6115  28.9286     23.44   
          TX               Melons        2.0 -99.6115  28.9286     23.44   
          TX               Onions        4.0 -99.6115  28.9286     23.44   
          TX              Spinach        5.0 -99.6115  28.9286     23.44   
          TX     Vegetable totals  6496740.0 -99.6115  28.9286     23.44   

                 Min_temp  
County    State            
Abbeville SC         3.21  
          SC         3.21  
          SC         3.21  
          SC         3.21  
          SC         3.21  
...                   ...  
Zavala    TX        19.42  
          TX        19.42  
          TX        19.42  
          TX        19.42  
          TX        19.42  

[49489 rows x 6 columns]

In [28]:
#reseting index
vegetables_merged.reset_index(inplace=True)

In [29]:
#All vegetables names
vegetables_list=vegetables_merged['Commodity'].unique().tolist()
vegetables_list

['Beans',
 'Broccoli',
 'Cabbage',
 'Carrots',
 'Cauliflower',
 'Cucumbers',
 'Eggplant',
 'Greens',
 'Melons',
 'Okra',
 'Peas',
 'Peppers',
 'Potatoes',
 'Pumpkins',
 'Radishes',
 'Squash',
 'Sweet corn',
 'Sweet potatoes',
 'Tomatoes',
 'Turnips',
 'Vegetable totals',
 'Asparagus',
 'Beets',
 'Herbs',
 'Lettuce',
 'Onions',
 'Parsley',
 'Spinach',
 'Vegetables, other',
 'Artichokes',
 'Brussels sprouts',
 'Garlic',
 'Rhubarb',
 'Celery',
 'Daikon',
 'Horseradish',
 'Ginger root',
 'Chicory',
 'Escarole & endive',
 'Watercress',
 'Ginseng']

In [30]:
#saving combined data as csv 
vegetables_merged.to_csv("cleaned_data/vegetables_merged.csv",encoding="utf-8", index=False)

In [31]:
#saving data as csv 
vegetables_df.to_csv("cleaned_data/vegetables_data.csv",encoding="utf-8", index=False)

## Fruits

In [32]:
#reading all fruits csv for different states data to pandas dataframe
fruits_data=[]
for i in range(1,8):
    df=pd.read_csv(f'data/fruits{i}.csv')
    fruits_data.append(df)
#concating all states datas to one dataframe
fruits=pd.concat([data for data in fruits_data], axis=0)

In [33]:
#selecting only important columns
fruits=fruits[['County','State','Commodity', 'Data Item', 'Domain', 'Domain Category', 'Value']]

In [34]:
#replacing string (D) with 0 in value columns and converting to float
fruits['Value']=fruits['Value'].replace({' (D)':0, ' (Z)':0})
fruits['Value']=fruits['Value'].str.replace(',', '').astype(float)

In [35]:
#value for each comodity with county
fruits_df=fruits.groupby(['County', 'State', 'Commodity']).agg({'Value':sum})
fruits_df.reset_index(inplace=True)
fruits_df

,County,State,Commodity,Value
0,ABBEVILLE,SOUTH CAROLINA,APPLES,3.0
1,ABBEVILLE,SOUTH CAROLINA,BERRY TOTALS,85054.0
2,ABBEVILLE,SOUTH CAROLINA,BLUEBERRIES,84.0
3,ABBEVILLE,SOUTH CAROLINA,CHESTNUTS,2.0
4,ABBEVILLE,SOUTH CAROLINA,FRUIT & TREE NUT TOTALS,159021.0
...,...,...,...,...
41060,YUMA,ARIZONA,TANGERINES,12.0
41061,ZAVALA,TEXAS,FRUIT & TREE NUT TOTALS,8.0
41062,ZAVALA,TEXAS,ORCHARDS,21.0
41063,ZAVALA,TEXAS,PECANS,68.0


In [36]:
fruits_df

,County,State,Commodity,Value
0,ABBEVILLE,SOUTH CAROLINA,APPLES,3.0
1,ABBEVILLE,SOUTH CAROLINA,BERRY TOTALS,85054.0
2,ABBEVILLE,SOUTH CAROLINA,BLUEBERRIES,84.0
3,ABBEVILLE,SOUTH CAROLINA,CHESTNUTS,2.0
4,ABBEVILLE,SOUTH CAROLINA,FRUIT & TREE NUT TOTALS,159021.0
...,...,...,...,...
41060,YUMA,ARIZONA,TANGERINES,12.0
41061,ZAVALA,TEXAS,FRUIT & TREE NUT TOTALS,8.0
41062,ZAVALA,TEXAS,ORCHARDS,21.0
41063,ZAVALA,TEXAS,PECANS,68.0


In [37]:
test = fruits_df['State'].unique()

In [38]:
test

array(['SOUTH CAROLINA', 'LOUISIANA', 'VIRGINIA', 'IDAHO', 'IOWA',
       'KENTUCKY', 'MISSOURI', 'OKLAHOMA', 'COLORADO', 'ILLINOIS',
       'INDIANA', 'MISSISSIPPI', 'NEBRASKA', 'OHIO', 'PENNSYLVANIA',
       'WASHINGTON', 'WISCONSIN', 'VERMONT', 'MINNESOTA', 'FLORIDA',
       'NORTH CAROLINA', 'CALIFORNIA', 'NEW YORK', 'WYOMING', 'MICHIGAN',
       'ALASKA', 'MARYLAND', 'KANSAS', 'TENNESSEE', 'TEXAS', 'MAINE',
       'ARIZONA', 'GEORGIA', 'ARKANSAS', 'NEW JERSEY', 'ALABAMA',
       'OREGON', 'WEST VIRGINIA', 'NORTH DAKOTA', 'MASSACHUSETTS',
       'SOUTH DAKOTA', 'UTAH', 'MONTANA', 'NEW HAMPSHIRE', 'NEW MEXICO',
       'RHODE ISLAND', 'NEVADA', 'CONNECTICUT', 'HAWAII', 'DELAWARE'],
      dtype=object)

In [39]:
test2 = fruits_df.loc[(fruits_df["State"]=="CA") & (fruits_df["Commodity"]=="Peaches")]

In [40]:
test2.head()

,County,State,Commodity,Value


In [41]:
test3 = fruits_df.groupby(['State', 'Commodity']).count()

In [42]:
test4 = test3.reset_index()

In [43]:
test4

,State,Commodity,County,Value
0,ALABAMA,ALMONDS,3,3
1,ALABAMA,APPLES,50,50
2,ALABAMA,APRICOTS,4,4
3,ALABAMA,ARONIA BERRIES,2,2
4,ALABAMA,"BERRIES, OTHER",2,2
...,...,...,...,...
1721,WYOMING,PEARS,3,3
1722,WYOMING,PLUMS,4,4
1723,WYOMING,PLUMS & PRUNES,4,4
1724,WYOMING,RASPBERRIES,8,8


In [44]:
test7 = fruits_df.groupby(['State',"County"]).count()

In [45]:
test7 = test7.groupby(["State"]).count()

In [46]:
test7

,Commodity,Value
State,,
ALABAMA,67,67
ALASKA,5,5
ARIZONA,14,14
ARKANSAS,73,73
CALIFORNIA,57,57
COLORADO,36,36
CONNECTICUT,8,8
DELAWARE,3,3
FLORIDA,67,67


In [47]:
test5 = test4.loc[(test4["Commodity"]=="Apples")]

In [48]:
test5

,State,Commodity,County,Value


In [49]:
test6 = test4.loc[(test4["State"]=="MA")]

In [50]:
len(test6)

0

In [51]:
#making state , county captilize
fruits_df['State']=fruits_df['State'].str.capitalize()
fruits_df['County']=fruits_df['County'].str.capitalize()
fruits_df['Commodity']=fruits_df['Commodity'].str.capitalize()

In [52]:
fruits_df['State'] = fruits_df['State']. apply(state_abbr)
fruits_df

,County,State,Commodity,Value
0,Abbeville,SC,Apples,3.0
1,Abbeville,SC,Berry totals,85054.0
2,Abbeville,SC,Blueberries,84.0
3,Abbeville,SC,Chestnuts,2.0
4,Abbeville,SC,Fruit & tree nut totals,159021.0
...,...,...,...,...
41060,Yuma,AZ,Tangerines,12.0
41061,Zavala,TX,Fruit & tree nut totals,8.0
41062,Zavala,TX,Orchards,21.0
41063,Zavala,TX,Pecans,68.0


In [53]:
#setting index county and state
df_fruits=fruits_df.set_index(["County", "State"])


In [54]:
#merging with weather to get lat and lon
fruits_merged = df_fruits.merge(df_weather, how='inner', left_index=True, right_index=True)

In [55]:
#reseting index
fruits_merged.reset_index(inplace=True)

In [56]:
#All fruits names
fruits_list=fruits_merged['Commodity'].unique().tolist()
fruits_list

['Apples',
 'Berry totals',
 'Blueberries',
 'Chestnuts',
 'Fruit & tree nut totals',
 'Hazelnuts',
 'Non-citrus totals',
 'Orchards',
 'Pears',
 'Pecans',
 'Strawberries',
 'Tree nut totals',
 'Berries, other',
 'Elderberries',
 'Figs',
 'Grapes',
 'Non-citrus, other',
 'Peaches',
 'Pomegranates',
 'Walnuts',
 'Apricots',
 'Blackberries',
 'Cherries',
 'Nectarines',
 'Plum-apricot hybrids',
 'Plums',
 'Plums & prunes',
 'Prunes',
 'Raspberries',
 'Aronia berries',
 'Persimmons',
 'Almonds',
 'Tree nuts, other',
 'Currants',
 'Kiwifruit',
 'Cranberries',
 'Boysenberries',
 'Avocados',
 'Citrus totals',
 'Grapefruit',
 'Guavas',
 'Kumquats',
 'Lemons',
 'Limes',
 'Olives',
 'Oranges',
 'Papayas',
 'Passion fruit',
 'Tangelos',
 'Tangerines',
 'Pistachios',
 'Loganberries',
 'Bananas',
 'Cherimoyas',
 'Macadamias',
 'Mangoes',
 'Pineapples',
 'Citrus, other',
 'Dates',
 'Coffee']

In [57]:
#saving combined data as csv 
fruits_merged.to_csv("cleaned_data/fruits_merged.csv",encoding="utf-8", index=False)

In [58]:
#saving data as csv 
fruits_df.to_csv("cleaned_data/fruits_data.csv",encoding="utf-8", index=False)

In [59]:
## All Crops

In [60]:
all_crops=['Corn',
 'Cotton',
 'Grain',
 'Hay',
 'Haylage',
 'Oats',
 'Peas',
 'Rye',
 'Soybeans',
 'Sunflower',
 'Wheat',
 'Barley',
 'Mint',
 'Rapeseed',
 'Sorghum',
 'Beans',
 'Chickpeas',
 'Grasses',
 'Legumes',
 'Mustard',
 'Sugarbeets',
 'Triticale',
 'Tobacco',
 'Herbs',
 'Millet',
 'Emmer & spelt',
 'Popcorn',
 'Canola',
 'Flaxseed',
 'Maple syrup',
 'Buckwheat',
 'Camelina',
 'Hops',
 'Peanuts',
 'Wild rice',
 'Sugarcane',
 'Rice',
 'Sesame',
 'Taro',
 'Safflower',
 'Guar',
 'Miscanthus',
 'Lentils',
 'Switchgrass',
 'Dill',
 'Jojoba',
'Beans',
 'Broccoli',
 'Cabbage',
 'Carrots',
 'Cauliflower',
 'Cucumbers',
 'Eggplant',
 'Greens',
 'Melons',
 'Okra',
 'Peas',
 'Peppers',
 'Potatoes',
 'Pumpkins',
 'Radishes',
 'Squash',
 'Sweet corn',
 'Sweet potatoes',
 'Tomatoes',
 'Turnips',
 'Asparagus',
 'Beets',
 'Herbs',
 'Lettuce',
 'Onions',
 'Parsley',
 'Spinach',
 'Artichokes',
 'Brussels sprouts',
 'Garlic',
 'Rhubarb',
 'Celery',
 'Daikon',
 'Horseradish',
 'Ginger root',
 'Chicory',
 'Escarole & endive',
 'Watercress',
 'Ginseng',
'Apples',
 'Berry totals',
 'Blueberries',
 'Chestnuts',
 'Hazelnuts',
 'Non-citrus totals',
 'Orchards',
 'Pears',
 'Pecans',
 'Strawberries',
 'Tree nut totals',
 'Berries, other',
 'Elderberries',
 'Figs',
 'Grapes',
 'Non-citrus, other',
 'Peaches',
 'Pomegranates',
 'Walnuts',
 'Apricots',
 'Blackberries',
 'Cherries',
 'Nectarines',
 'Plum-apricot hybrids',
 'Plums',
 'Plums & prunes',
 'Prunes',
 'Raspberries',
 'Aronia berries',
 'Persimmons',
 'Almonds',
 'Currants',
 'Kiwifruit',
 'Cranberries',
 'Boysenberries',
 'Avocados',
 'Citrus totals',
 'Grapefruit',
 'Guavas',
 'Kumquats',
 'Lemons',
 'Limes',
 'Olives',
 'Oranges',
 'Papayas',
 'Passion fruit',
 'Tangelos',
 'Tangerines',
 'Pistachios',
 'Loganberries',
 'Bananas',
 'Cherimoyas',
 'Macadamias',
 'Mangoes',
 'Pineapples',
 'Citrus',
 'Dates',
 'Coffee']
    

In [61]:
all_crops=[ i.capitalize() for i in all_crops]